<a href="https://colab.research.google.com/github/AliKarimiENT/NLP_HW1/blob/main/NLPHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [2]:
pip install hazm

     |████████████████████████████████| 316 kB 5.4 MB/s 
     |████████████████████████████████| 1.4 MB 43.2 MB/s 
     |████████████████████████████████| 233 kB 50.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=4aa2857e52787bee1ed79a131927d569b6af510fbdccd20903e36f345ecdd036
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154451 sha256=fa17d91e68c0262694c37b48180627c4f42c34a69d1ac99d3d822cc994654415
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


Reading txt file and getting verses

Here we need to do some processes

In [6]:
# with this function we check that it contains number in the text or not
def hasNumbers(inputString) :
  for character in inputString :
    if character.isdigit() :
      return True
  return False

def hasDash(inputString) :
  for character in inputString :
    if character == '-' :
      return True
  return False 


with open('/content/ShamsDaftar6.doc_filename=ShamsDaftar6.txt') as f:
  dataSet = f.readlines()

verses = []
for line in dataSet :
  if line != ' ' and line != '\n' and hasNumbers(line) == False  and hasDash(line)==False:
    if dataSet.index(line) != 0 :
      verses.append(line.replace('\t',' '))


# for i in range(20):
#   print(verses[i])


Get list of stop words

In [12]:
import re
with open('/content/stopwords.txt') as f:
  stops = f.readlines()
stopWords = []
for stop in stops :
  stopWords.append(re.sub('\n','',stop.replace('\ufeff','')))
print(stopWords)

['!', '"', '#', '(', ')', '*', ',', '-', '.', '/', ':', '[', ']', '«', '»', '،', '؛', '؟اتفاقا', 'اتفاقا', 'احتراما', 'احتمالا', 'اري', 'اری', 'آری', 'آري', 'از', 'ازجمله', 'اساسا', 'است', 'اش', 'اشكارا', 'آشكارا', 'اصلا', 'اصولا', 'اغلب', 'اكثرا', 'اكنون', 'الان', 'الآن', 'البته', 'ام', 'اما', 'امد', 'آمد', 'امدم', 'آمد', 'امدن', 'آمدن', 'امدند', 'آمدند', 'امده', 'آمده', 'امدي', 'آمدی', 'امديد', 'آمدید', 'امديم', 'آمدیم', 'امروزه', 'امسال', 'امشب', 'ان', 'آن', 'اند', 'انشاالله', 'انصافا', 'انطور', 'آنطور', 'انقدر', 'آنقدر', 'انها', 'آنها', 'انچنان', 'آنچنان', 'انگار', 'او', 'اورد', 'آورد', 'اوردم', 'آوردم', 'اوردن', 'آوردن', 'اوردند', 'آوردند', 'اورده', 'آورده', 'اوردي', 'اوردی', 'آوردی', 'اورديد', 'آوردید', 'اورديم', 'آوردیم', 'اورم', 'آورم', 'اورند', 'آورند', 'اوري', 'اوری', 'آوری', 'اوريد', 'آورید', 'اوريم', 'آوری', 'اولا', 'اي', 'آی', 'ای', 'ايا', 'آیا', 'ايد', 'ايشان', 'ايم', 'اين', 'ايند', 'آيند', 'اينطور', 'اينقدر', 'اينك', 'اینک', 'اينها', 'اينچنين', 'اينگونه', 'ايي', 'ایی', '

Now it's time to use **Hazm** **Library** to tokenize each sentence

In [19]:
from hazm import word_tokenize
concept_words = []
for verse in verses:
  # now we must add all words in the list to words list
  for word in word_tokenize(verse):
    concept_words.append(word)

Remove stop words from verses

In [22]:
from os import remove
# for removing stop words from our words list we need to this process
corpus  = [] #concept words list is a list that contains list of words without stopWords
for word in concept_words :
 if word not in stopWords:
   corpus .append(word)
 
print(corpus)

['گرم', 'سیم', 'درم', 'مرا', 'مونس', 'کم', 'وگر', 'یارم', 'فقیرستی', 'زر', 'فارغ', 'غم', 'خدایا', 'حرمت', 'مردان', 'دنیا', 'فارغش', 'گردان', 'فارغستی', 'کم', 'نگارا', 'مرا', 'خواهی_وگر', 'همدرد', 'همراهی', 'مکن', 'مخور', 'حسرت', 'بختم', 'محتشم', 'بتا', 'زیبا', 'نیکویی', 'رها', 'کن', 'این', 'گدارویی', 'چشم', 'سیرستی', 'فلک', 'حشم', 'طمع', 'آدمی', 'خویش', 'بیگانه', 'وگر', 'طمع', 'خال', 'عم', 'بیا', 'شو', 'مه', 'نعمت', 'جو', 'دولت', 'جو', 'ابلیس', 'این', 'شه', 'صاحب', 'علم', 'ابلیسی', 'سقط', 'ثنا', 'جفا', 'وفا', 'سقم', 'کرم', 'زهی', 'اقبال', 'درویشی', 'زهی', 'اسرار', 'خویشی', 'دانستیی', 'پیشت', 'عدم', 'جهانی', 'هیچان', 'خیال', 'خواب', 'پیچان', 'وگر', 'خفته', 'بدانستی', 'خوابم', 'غم', 'خیالی', 'بیند', 'این', 'خفته', 'اندیشه', 'فرورفته', 'وگر', 'خواب', 'آشفته', 'بجستی', 'نعم', 'زندان', 'غم', 'دیده', 'باغ', 'ارم', 'دیده', 'وگر', 'بیدار', 'گشتی', 'زندان', 'نی', 'ارم', 'امیر', 'دل', 'گوید', 'دلی', 'عاشق', 'گیری', 'نان', 'جامه', 'بیزاری', 'قحط', 'نان', 'کند', 'عشق', 'خبازی', 'وگر', 'گم', 'گشت',



**Tokenize** corpus vocabulary

Import keras and use it 

In [25]:
from tensorflow import keras
from keras.preprocessing import text
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus)
wordToIndex = tokenizer.word_index
wordToIndex = {v:k for k,v in wordToIndex.items()}
print(wordToIndex)

{1: 'این', 2: 'جان', 3: 'دل', 4: 'عشق', 5: 'سر', 6: 'نی', 7: 'کن', 8: 'شمس', 9: 'صد', 10: 'خوش', 11: 'جهان', 12: 'چشم', 13: 'ای', 14: 'دست', 15: 'آب', 16: 'دم', 17: 'آتش', 18: 'مرا', 19: 'کند', 20: 'عالم', 21: 'شب', 22: 'زان', 23: 'مه', 24: 'نیست', 25: 'آید', 26: 'نور', 27: 'عقل', 28: 'گل', 29: 'کردی', 30: 'دیده', 31: 'بس', 32: 'تن', 33: 'روز', 34: 'خاک', 35: 'تویی', 36: 'ماه', 37: 'تبریز', 38: 'غم', 39: 'جمله', 40: 'خانه', 41: 'خویش', 42: 'مست', 43: 'آخر', 44: 'ست', 45: 'آنک', 46: 'کو', 47: 'ره', 48: 'لطف', 49: 'میان', 50: 'پر', 51: 'دیدی', 52: 'جانی', 53: 'همچو', 54: 'شکر', 55: 'عاشق', 56: 'بحر', 57: 'خون', 58: 'باری', 59: 'شاه', 60: 'یار', 61: 'خورشید', 62: 'شه', 63: 'ساقی', 64: 'گشتی', 65: 'تبریزی', 66: 'زهی', 67: 'دانی', 68: 'وگر', 69: 'باده', 70: 'چرخ', 71: 'خدا', 72: 'شو', 73: 'مستی', 74: 'باد', 75: 'گوید', 76: 'دریا', 77: 'رفتی', 78: 'پنهان', 79: 'پرده', 80: 'پای', 81: 'باغ', 82: 'زر', 83: 'روح', 84: 'رخ', 85: 'کرده', 86: 'کار', 87: 'بیا', 88: 'برون', 89: 'عجب', 90: 'هله', 91: 